1. Load up the 1K jeopardy dataset that has 1000 objects in total, keep at least the question, answer and round properties.
2. How do you check for the number of objects stored in the database?
3. Search for objects that are close to the concept of “spicy food recipes” and show 4 QnA
4. Can you find “spicy food recipes” related questions that were used in Double Jeopardy rounds?


### Q1: Load up the dataset,  keep at least the question, answer and round properties.

In [1]:
import requests
import json

# Download the data
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/intro-workshop/main/data/jeopardy_1k.json')
data = json.loads(resp.text)  # Load data

# Parse the JSON and preview it
print(type(data), len(data))
print(json.dumps(data[1], indent=2))

<class 'list'> 1000
{
  "Air Date": "2005-11-18",
  "Round": "Jeopardy!",
  "Value": 200,
  "Category": "RHYME TIME",
  "Question": "Any pigment on the wall so faded you can barely see it",
  "Answer": "faint paint"
}


In [2]:
import weaviate
from weaviate import EmbeddedOptions
import os

client = weaviate.Client(
    embedded_options=EmbeddedOptions(),
    additional_headers={
        "X-OpenAI-Api-Key": os.environ["OPENAI_API_KEY"]  # Replace this with your actual key
    }
)

embedded weaviate is already listening on port 6666


In [3]:
if client.schema.exists("Question"):
    client.schema.delete_class("Question")

In [4]:
#Define the class that will be used to add the data
# We need to have properties for the questions, answer and round

class_definition = {
    
    "class": "Question",
    "vectorizer":"text2vec-openai",
    "vectorIndexConfig": {
        "distance" : "cosine"
    },
    
    'properties' : [
        {
            'name' : "question",
            "dataType" : ['text']
        },
        {
            'name' : "answer",
            "dataType" : ["text"]
        },
        {
            'name' : 'round',
            'dataType': ['text']
        }
    ]
}

client.schema.create_class(class_definition)

In [5]:
#Insert the data into Weaviate


with client.batch() as batch:
    for o in data:
        obj_body = {
            'question':o["Question"],
            'answer':o["Answer"],
            'round':o["Round"]
        }
        
        batch.add_data_object(
        data_object=obj_body,
        class_name="Question"
        )

/Users/linkedin/anaconda3/lib/python3.11/site-packages/weaviate/warnings.py:130: DeprecationWarning: Dep006: You are using the `client.batch()` method, which will be removed in the next major release.
            Please instead use the `client.batch.configure()` method to configure your batch and `client.batch` to enter the context manager.
            See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


### Q2. How do you check for the number of objects stored in the database?

In [6]:
print(json.dumps(client.query.aggregate("Question").with_meta_count().do(), indent=2))

{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 1000
          }
        }
      ]
    }
  }
}


### 3. Search for objects that are close to the concept of "spicy food recipes" and show 4 QnA

In [7]:
response = (client.query
            .get("Question", ['question','answer'])
            .with_near_text({"concepts":"spicy food recipes"})
            .with_additional(['distance'])
            .with_limit(4)
            .do()
)

print(json.dumps(response, indent=2))

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "distance": 0.20050466
          },
          "answer": "tripe",
          "question": "Popular in Pennsylvania, pepper pot is a peppery soup made from this stomach lining"
        },
        {
          "_additional": {
            "distance": 0.20277321
          },
          "answer": "Chiles Rellenos",
          "question": "The name of this Mexican dish made with chiles & cheese translates to \"stuffed peppers\""
        },
        {
          "_additional": {
            "distance": 0.20874399
          },
          "answer": "Truffles",
          "question": "Dishes flavored or garnished with these rare costly fungi are referred to as \"a la Perigourdine\""
        },
        {
          "_additional": {
            "distance": 0.21527755
          },
          "answer": "bread",
          "question": "Naan, an east Indian form of this, is usually baked in a tandoor"
        }
      ]

### 4. Can you find "spicy food recipes" related questions that were used in Double Jeopardy rounds?

In [9]:
response = (client.query
            .get("Question", ['question','answer', 'round'])
            .with_near_text({"concepts":"spicy food recipes"})
            .with_additional(['distance'])
            .with_limit(3)
            .with_where({
                "path": ['round'],
                "operator" : "Equal",
                "valueText" : "Double Jeopardy!"
            })
            .do()
)

print(json.dumps(response, indent=2))

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "distance": 0.22439378
          },
          "answer": "\"Jambalaya\"",
          "question": "This Creole concoction of meat & seafood is so good Hank Williams wrote a song about it in 1952",
          "round": "Double Jeopardy!"
        },
        {
          "_additional": {
            "distance": 0.23735112
          },
          "answer": "Herring",
          "question": "The Bismarck type of this fish is made of fillets cured in vinegar, salt & onions",
          "round": "Double Jeopardy!"
        },
        {
          "_additional": {
            "distance": 0.24249423
          },
          "answer": "litmus",
          "question": "Vegetable dye that turns red in acid solutions & blue in alkaline solutions",
          "round": "Double Jeopardy!"
        }
      ]
    }
  }
}
